In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, round_
import pandas as pd
pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

/home/alberto/Docker/Trading/analysis/Functions.py:183: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events':
/home/alberto/Docker/Trading/analysis/Functions.py:183: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events':


In [2]:
start_interval=datetime(2023,5,7)
end_interval=datetime.now()

filter_position=(0,100)

data, df = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

StatusCode for getting most_traded_coins: 200
new Json saved in /home/alberto/Docker/Trading/tracker/json/most_traded_coins.json
['/home/alberto/Docker/Trading/analysis/json/data-8-5-2023-122449.json']
Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-8-5-2023-122449.json
92 coins have been loaded


,n_observations
AGIXUSDT,4775
DOGEUSDT,4775
ALGOUSDT,4775
MANAUSDT,4775
SHIBUSDT,4775
OGUSDT,4775
CTSIUSDT,4775
STXUSDT,4775
DOTUSDT,4775
MAGICUSDT,4775


In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m',]
list_minutes = [5, 15, 30]
list_event_buy_volume = [0.55, 0.60]
list_event_volume = [5,6]
resp = None
resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)
df = pd.DataFrame(resp).transpose()
df

In [3]:
# ANALYSIS MULTIPROCESSING 2
t1 = time()

data_arguments = data_preparation(data, n_slices = 6)

# MULTIPROCESSING WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

total_combinations = len(list_buy_vol) * len(list_vol) * len(list_minutes) * len(list_event_volume) * len(list_event_buy_volume)
print('total_combinantions', ': ', total_combinations)
path = "/home/alberto/Docker/Trading/analysis/analysis_json/"



now = datetime.now()
now_year = str(now.year)
now_month = str(now.month)
now_day = str(now.day)
now_hour = str(now.hour)
now_minute = str(now.minute)
file_path = path + now_month+now_day+ '_' + now_hour+now_minute + '.json'

manager = Manager()
# Create shared memory for JSON data
shared_data = manager.Value(str, json.dumps({}))
lock = Manager().Lock()


# Create a multiprocessing Pool
pool = Pool()

# Execute the function in parallel
pool.starmap(wrap_analyze_events_multiprocessing, [(arg, arg_i, list_buy_vol, list_vol, list_minutes,
                                    list_event_buy_volume, list_event_volume, start_interval,
                                    end_interval, file_path, lock, shared_data) for arg, arg_i in zip(data_arguments, range(1,len(data_arguments)+1))])

# Close the pool
pool.close()
pool.join()
t2 = time()
print(t2-t1, ' seconds')


92 coins will be analyzed from 2023-05-07T00:00:05.289235 to 2023-05-10T07:34:05.171656
total_combinantions :  448


/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret

23.615497827529907  seconds


In [ ]:
shared_data.value

In [4]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
lb = []
for index, row in df.iterrows():
    mean = row['mean']
    std = row['std']
    lb.append(mean - std)
    
    
df['lb'] = lb
    
df.sort_values(by=['lb'], ascending=False)

,mean,std,n_coins,n_events,lb
buy_vol_15m:0.7/vol_30m:6/timeframe:5,11.527971,1.802103,5.0,10.0,9.725868
buy_vol_15m:0.7/vol_30m:6/timeframe:180,9.934120,2.416055,5.0,7.0,7.518064
buy_vol_15m:0.7/vol_30m:6/timeframe:30,9.626999,2.202374,5.0,8.0,7.424625
buy_vol_15m:0.7/vol_30m:6/timeframe:15,9.676230,2.304528,5.0,8.0,7.371702
buy_vol_15m:0.7/vol_30m:6/timeframe:60,9.418049,2.192928,5.0,8.0,7.225122
buy_vol_15m:0.7/vol_30m:6/timeframe:120,9.274868,2.176873,5.0,8.0,7.097996
buy_vol_15m:0.7/vol_30m:6/timeframe:360,6.254454,2.661171,5.0,6.0,3.593283
buy_vol_30m:0.7/vol_60m:5/timeframe:360,3.555674,0.000000,1.0,2.0,3.555674
buy_vol_30m:0.7/vol_60m:6/timeframe:360,3.203704,0.000000,1.0,1.0,3.203704
buy_vol_30m:0.7/vol_60m:5/timeframe:180,2.726021,0.000000,1.0,2.0,2.726021


In [ ]:
df = pd.DataFrame(resp).transpose()
df

In [ ]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df